In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [6]:
# Veriyi yükle
data_dir = './Trafik'  # Görüntü dosyalarının bulunduğu klasör
csv_file = './labelss.csv'      # CSV dosyasının yolu

# CSV dosyasını oku
df = pd.read_csv(csv_file)

# Görüntü dosyalarını ve etiketlerini yükle
images = []
labels = []
for index, row in df.iterrows():
    class_id = row['ClassId']
    class_name = row['Name']
    # Görüntü dosya yolunu oluştur
    image_path = os.path.join(data_dir, str(class_id))
    for image_file in os.listdir(image_path):
        image = load_img(os.path.join(image_path, image_file), target_size=(32, 32))
        image = img_to_array(image)
        images.append(image)
        labels.append(class_id)

# NumPy array'lerine dönüştür
X = np.array(images, dtype='float32') / 255.0
y = np.array(labels, dtype='int')

# Etiketleri one-hot encode et
y = to_categorical(y, num_classes=58)  # 43 sınıf olduğu varsayılıyor, gerekirse ayarlayın

# Eğitim ve test verilerini ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Veri artırma (data augmentation) için ImageDataGenerator kullan
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)


In [7]:
# Modeli oluştur
model = Sequential()

# İlk Convolutional Layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# İkinci Convolutional Layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Üçüncü Convolutional Layer
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten Layer
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(58, activation='softmax'))

# Modeli derle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model.fit(datagen.flow(X_train, y_train, batch_size=8), validation_data=(X_test, y_test), epochs=20)

c:\Users\alii2\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
  24/1558 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.0432 - loss: 4.0239    

c:\Users\alii2\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1558/1558 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.0857 - loss: 3.5225 - val_accuracy: 0.6275 - val_loss: 1.2884
Epoch 2/20
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5656 - loss: 1.4539 - val_accuracy: 0.8505 - val_loss: 0.5217
Epoch 3/20
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7630 - loss: 0.7695 - val_accuracy: 0.9313 - val_loss: 0.2286
Epoch 4/20
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8402 - loss: 0.5008 - val_accuracy: 0.9573 - val_loss: 0.1420
Epoch 5/20
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.8753 - loss: 0.3933 - val_accuracy: 0.9692 - val_loss: 0.0931
Epoch 6/20
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8997 - loss: 0.3089 - val_accuracy: 0.9724 - val_loss: 0.0952
Epoch 7/20
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9100 - loss: 0.2768 - val_accuracy: 0.9660 - val_loss: 0.1028
Epoch 8/20
1558/1558 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9272 - loss: 0.2278 - val_accurac

In [8]:
model.save("chatmodel4.h5")
